In [0]:
#confirm spark or create one
try:
    print("spark exists. Spark version:", spark.version)
except NameError:
    from pyspark.sql import SparkSession
    spark = SparkSession.builder.appName("pyspark-scaffold-day1").getOrCreate()
    print("Created SparkSession. Spark version:", spark.version)


In [0]:
# - read sample dataset
df = spark.read.csv("/databricks-datasets/retail-org/customers/customers.csv",
                    header=True, inferSchema=True)

df.printSchema()
df.show(5)
print("Row count (sample):", df.count())


In [0]:
# Show all column names
print(df.columns)

# Show row count & summary stats for numeric columns
#df.describe().show()


In [0]:
df.select("customer_id", "customer_name", "city", "state").show(5)

df2 = df.withColumnRenamed("customer_id", "cust_id")
df2.select("cust_id", "customer_name").show(5)


In [0]:
from pyspark.sql.functions import col, length, upper

# length of customer name
df3 = df.withColumn("name_length", length(col("customer_name")))

# uppercase city
df3 = df3.withColumn("city_upper", upper(col("city")))

df3.select("customer_name", "city", "city_upper", "name_length").show(5)


In [0]:
# customers from California
df.filter(col("state") == "CA").show(5)

# customers with more than 100 units purchased
df.filter(col("units_purchased") > 100).select("customer_name", "units_purchased").show(5)


In [0]:
df.orderBy(col("units_purchased").desc()).select("customer_name", "units_purchased").show(10)


In [0]:
# distinct states
df.select("state").distinct().show()

# count customers per state
df.groupBy("state").count().orderBy(col("count").desc()).show(10)
